In [5]:
from gcopy.my_copy import (
    Code,
    get_implicit_iterator_in_for_loop_everywhere,
    get_implicit_iterator_in_for_loop,
)
import inspect

### jupyter use to quick tests

In [6]:
def my_copy(generator):
    code = Code.from_generator(generator)
    args = (
        generator.gi_frame.f_locals.copy()
    )  # need to pass all globals that may be used as well.
    args = ",".join(f"{k}={v}" for k, v in args.items())
    return _my_copy(code, args)


def _my_copy(code, args, current=None):
    # function def, not really needed, but fine

    new_code = current if current else Code(f"def saved({args}):\n")
    line = code.running_line + 1

    # start with the most extenral block
    if code.block in ["if", "else:", "elif"]:
        # if if is inside a while
        new_code += "    if True:"
        new_code += "        pass"
    elif code.block == "while":
        # If while is found the rest of the iteration goes to the top. as is the starting point
        new_code += code.scope_after.dedent.indent
        line = code.block_start_line
    elif code.block == "for":
        # worst case as we need to capture the iterator with magic
        new_code += code.scope_after.dedent.indent
        new_code += "for NOTIMPLEMENTED"
        line = code.block_start_line + 1

    # main code
    new_code += code[line:]

    # executor
    new_code += f"n = saved({args})"
    print(new_code.code)
    exec(new_code.code)
    return eval("n")

In [7]:
def test(a):
    a = a + 2
    yield a
    while True:
        a += 1
        if a > 5:
            yield a
        a += 2

In [8]:
def my_copy(generator):
    code = Code.from_generator(generator)
    args = (
        generator.gi_frame.f_locals.copy()
    )  # need to pass all globals that may be used as well.
    args = ",".join(f"{k}={v}" for k, v in args.items())
    return _my_copy(code, args)


def _my_copy(code, args, current=None):
    # function def, not really needed, but fine

    new_code = current if current else Code(f"def saved({args}):\n")
    line = code.running_line + 1

    # start with the most extenral block
    if code.block in ["if", "else:", "elif"]:
        # if if is inside a while
        new_code += "    if True:"
        new_code += "        pass"
    elif code.block == "while":
        # If while is found the rest of the iteration goes to the top. as is the starting point
        new_code += code.scope_after.dedent.indent
        line = code.block_start_line
    elif code.block == "for":
        # worst case as we need to capture the iterator with magic
        new_code += code.scope_after.dedent.indent
        new_code += "for NOTIMPLEMENTED"
        line = code.block_start_line + 1

    # main code
    new_code += code[line:]

    # executor
    new_code += f"n = saved({args})"
    print(new_code.code)
    exec(new_code.code)
    return eval("n")

In [9]:
t1 = test(10)
next(t1)
next(t1)

13

In [10]:
code = Code.from_generator(t1)

In [11]:
code

def test(a):
    a = a + 2
    yield a
    while True:
        a += 1
        if a > 5:
            yield a
        a += 2